In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import Counter, defaultdict

In [2]:
import sys
import os
def add_sys_path(p):
    p = os.path.abspath(p)
    if p not in sys.path:
        sys.path.append(p)

add_sys_path('..')

In [3]:
import evaluate

In [4]:
import data_split

In [5]:
from evaluate import read_dataset

In [6]:
import json

In [7]:
from tqdm.auto import tqdm, trange

# Чтение и сплит данных

In [8]:
n_ds = read_dataset('../data/training_data/training_nouns.tsv',  lambda x: json.loads(x))

In [9]:
len(n_ds)

25376

In [11]:
train, dev, test1, test2, hid, forbidden_words = data_split.split_dict(n_ds)

# Применение бейзлайна

In [12]:
import my_knn
from importlib import reload
reload(my_knn)

<module 'my_knn' from 'C:\\Users\\ddale\\YandexDisk\\code\\NLP\\taxonomy-enrichment\\experiments\\my_knn.py'>

In [14]:
import gensim
# this is araneum 2018 ft model from rusvectores
ft = gensim.models.fasttext.FastTextKeyedVectors.load(
    'C:/Users/ddale/Downloads/NLP/rusvectores/model.model'
)

In [15]:
ft_embedder = my_knn.SentenceEmbedder(ft=ft, n=300, normalize_word=True, pos_weights={'INFN': 1.0, 'PREP': 0.1})
print(ft_embedder('привет как дела').shape)

(300,)


In [16]:
taiga = gensim.models.KeyedVectors.load_word2vec_format(
    'C:/Users/ddale/Downloads/NLP/rusvectores/taiga_skipgram/model.bin', 
    binary=True,
)

In [17]:
w2v_embedder = my_knn.W2VWrapper(taiga)

In [18]:
syns_storage, rel_storage, rel_df = my_knn.prepare_storages(
    synsets_filename='../data/ruwordnet/synsets.N.xml',
    relations_filename='../data/ruwordnet/synset_relations.N.xml',
    forbidden_words=forbidden_words,
)

number of texts: 86549
forbidden senses are 4126
numer of ids 29296 long list is 95119



23827
466
5
30877


In [20]:
vecs = np.stack([ft_embedder(t) for t in tqdm(syns_storage.texts_long) ])

from sklearn.neighbors import KDTree
tree = KDTree(vecs)

In [44]:
w2v_vecs = np.stack([w2v_embedder(t) for t in tqdm(syns_storage.texts_long) ])
w2v_tree = KDTree(w2v_vecs)

In [45]:
hypos = my_knn.hypotheses_knn('брюки', index=tree, text2vec=ft_embedder, synset_storage=syns_storage, rel_storage=rel_storage)
hypos

['108194-N', '4024-N']

In [46]:
for id in hypos:
    print(syns_storage.id2synset[id]['@ruthes_name'])

ШТАНЫ, БРЮКИ
ВЕРХНЯЯ ОДЕЖДА


In [47]:
# проверка, что сравнение само с собой всегда получает скор 1
dev_pred = {k: [v0 for val in vals for v0 in val] for k, vals in dev.items()}
mean_ap, mean_rr = evaluate.get_score(dev, dev_pred, k=10)
print(mean_ap, mean_rr)

0.9996719160104987 1.0


In [62]:
def w2v_scorer(text1, text2):
    return np.dot(w2v_embedder.get_text_vec(text1), w2v_embedder.get_text_vec(text2)) ** 5

In [60]:
#texts = ttest_test1.one_text.drop_duplicates()
texts = sorted(dev.keys())
small_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        #index=tree, text2vec=ft_embedder, 
        index=w2v_tree, text2vec=w2v_embedder, 
        synset_storage=syns_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
    )  
    for txt in tqdm(texts)
}


In [61]:
mean_ap, mean_rr = evaluate.get_score(dev, small_hypos, k=10)
print(mean_ap, mean_rr)

0.4661363944090322 0.5028019935008124


Типовой метод получает скор 26/29 на публичном тесте, и 39/42 - на дев. сете. Выглядит как непорядок. 

```
0.39642414489855415 0.428703443319584  - baseline
0.4302434851893513 0.4609392575928008  - baseline with s^2 w2v penalty 
0.43823949089697095 0.471559023872015  - with s^3 penalty
0.44413862329708753 0.478044150731158  - with s^5 penalty
0.4578544869391327 0.4919471003624545  - w2v only with s^3 penalty
0.4660694756905386 0.5044838145231845* - w2v only with s^5 penalty 
0.4661363944090322 0.5028019935008124  - w2v only with s^10 penalty 
```

# Оценка применения

In [63]:
public_test_verbs = pd.read_csv('../data/public_test/nouns_public.tsv', header=None)
public_test_verbs.columns = ['text']

In [27]:
full_syn_storage, full_rel_storage, full_rel_df = my_knn.prepare_storages(
    synsets_filename='../data/ruwordnet/synsets.N.xml',
    relations_filename='../data/ruwordnet/synset_relations.N.xml',
    forbidden_words=set()
)

number of texts: 86549
forbidden senses are 0
numer of ids 29296 long list is 119273



28118
555
5
37249


In [28]:
full_ft_vecs = np.stack([ft_embedder(t) for t in tqdm(full_syn_storage.texts_long) ])
full_ft_tree = KDTree(full_ft_vecs)

In [64]:
full_w2v_vecs = np.stack([w2v_embedder(t) for t in tqdm(full_syn_storage.texts_long) ])
full_w2v_tree = KDTree(full_w2v_vecs)

In [65]:
public_test_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=full_w2v_tree, text2vec=w2v_embedder, 
        synset_storage=full_syn_storage, rel_storage=full_rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
    )  
    for txt in tqdm(public_test_verbs.text)
}

In [66]:
sub = my_knn.dict2submission(public_test_hypos, syns_storage.id2synset)
sub

,noun,result,result_text
0,АБДОМИНОПЛАСТИКА,616-N,БОЛЕЗНЬ
1,АБДОМИНОПЛАСТИКА,3406-N,ПОВРЕЖДЕНИЕ ЗДОРОВЬЯ
2,АБДОМИНОПЛАСТИКА,106567-N,ФИЗИЧЕСКОЕ САМОЧУВСТВИЕ
3,АБДОМИНОПЛАСТИКА,106665-N,"НАРУШИТЬ СОСТОЯНИЕ, ХОД"
4,АБДОМИНОПЛАСТИКА,1047-N,МЕДИЦИНА
...,...,...,...
7615,ЯТАГАН,5900-N,МЕТАЛЛИЧЕСКОЕ ИЗДЕЛИЕ
7616,ЯТАГАН,784-N,ОРУЖИЕ
7617,ЯТАГАН,106553-N,"ПРИСПОСОБЛЕНИЕ, ИНСТРУМЕНТ"
7618,ЯТАГАН,112933-N,"ОТВЕТИТЬ, ПОСТУПИТЬ В ОТВЕТ"


In [67]:
sub.to_csv('results/nouns_v3_w2v_only_simp5_penalty.tsv', sep='\t', encoding='utf-8', header=None, index=None)
sub.head(15)

,noun,result,result_text
0,АБДОМИНОПЛАСТИКА,616-N,БОЛЕЗНЬ
1,АБДОМИНОПЛАСТИКА,3406-N,ПОВРЕЖДЕНИЕ ЗДОРОВЬЯ
2,АБДОМИНОПЛАСТИКА,106567-N,ФИЗИЧЕСКОЕ САМОЧУВСТВИЕ
3,АБДОМИНОПЛАСТИКА,106665-N,"НАРУШИТЬ СОСТОЯНИЕ, ХОД"
4,АБДОМИНОПЛАСТИКА,1047-N,МЕДИЦИНА
5,АБДОМИНОПЛАСТИКА,108862-N,ВОСПАЛЕНИЕ (БОЛЕЗНЕННЫЙ ПРОЦЕСС)
6,АБДОМИНОПЛАСТИКА,5804-N,КАРДИОЛОГИЯ
7,АБДОМИНОПЛАСТИКА,1064-N,ХИРУРГИЯ
8,АБДОМИНОПЛАСТИКА,2977-N,МЕДИЦИНСКОЕ ОБОРУДОВАНИЕ
9,АБДОМИНОПЛАСТИКА,6129-N,СЕРДЕЧНО-СОСУДИСТЫЕ ЗАБОЛЕВАНИЯ


# Финальная заливка

In [48]:
private_test_verbs = pd.read_csv('../data/private_test/nouns_private.tsv', header=None)
private_test_verbs.columns = ['text']

In [44]:
private_test_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        index=tree, text2vec=embedder, synset_storage=syns_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
    )  
    for txt in tqdm(private_test_verbs.text)
}

In [45]:
sub = my_knn.dict2submission(private_test_hypos, syns_storage.id2synset)

In [46]:
sub

,noun,result,result_text
0,АБСЕНТЕИЗМ,119563-N,МИРОВОЗЗРЕНИЕ
1,АБСЕНТЕИЗМ,132973-N,ФИЛОСОФСКОЕ УЧЕНИЕ
2,АБСЕНТЕИЗМ,121514-N,УЧЕНИЕ (СОВОКУПНОСТЬ ТЕОРЕТИЧ. ПОЛОЖЕНИЙ)
3,АБСЕНТЕИЗМ,106470-N,"УБЕЖДЕНИЯ, ВЗГЛЯДЫ"
4,АБСЕНТЕИЗМ,2201-N,РЕЛИГИЯ
...,...,...,...
15245,ЯШМА,106554-N,"ПРЕДМЕТ, ВЕЩЬ"
15246,ЯШМА,109444-N,УКРАСИТЬ
15247,ЯШМА,820-N,ВЕЩЕСТВО
15248,ЯШМА,143683-N,ПРИРОДНОЕ МИНЕРАЛЬНОЕ ОБРАЗОВАНИЕ


In [47]:
sub.to_csv('results/private_nouns_v0_strange.tsv', sep='\t', encoding='utf-8', header=None, index=None)
sub.head(15)

,noun,result,result_text
0,АБСЕНТЕИЗМ,119563-N,МИРОВОЗЗРЕНИЕ
1,АБСЕНТЕИЗМ,132973-N,ФИЛОСОФСКОЕ УЧЕНИЕ
2,АБСЕНТЕИЗМ,121514-N,УЧЕНИЕ (СОВОКУПНОСТЬ ТЕОРЕТИЧ. ПОЛОЖЕНИЙ)
3,АБСЕНТЕИЗМ,106470-N,"УБЕЖДЕНИЯ, ВЗГЛЯДЫ"
4,АБСЕНТЕИЗМ,2201-N,РЕЛИГИЯ
5,АБСЕНТЕИЗМ,108333-N,СТИЛЬ В ИСКУССТВЕ
6,АБСЕНТЕИЗМ,124135-N,ИДЕОЛОГИЯ
7,АБСЕНТЕИЗМ,111965-N,ИДЕАЛИЗМ
8,АБСЕНТЕИЗМ,121164-N,ИЗВРАЩЕНИЕ (ПРОТИВОЕСТЕСТВЕННОЕ ПОВЕДЕНИЕ)
9,АБСЕНТЕИЗМ,6639-N,ПОЛИТИЧЕСКОЕ ТЕЧЕНИЕ
